<a href="https://colab.research.google.com/github/SelvamRaju/Natural-Language-Processing-LSTM/blob/main/NLP_LSTM_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP-LSTM-Part-2

## Domain : Social Media Analytics

In [1]:
%tensorflow_version 2.x
import tensorflow
tensorflow.__version__

'2.8.0'

In [39]:
#Imports
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from sklearn.model_selection import train_test_split

In [3]:
# Drive mounting
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Declaring the working directory (Directory in Drive)
os.chdir('/content/drive/MyDrive/Colab Notebooks/Data/');

# 1. Read and explore the data


In [6]:
df = pd.read_json("Sarcasm_Headlines_Dataset.json",lines = True)
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [8]:
df.shape

(28619, 3)

In [7]:
# Printing sarcatic vs non sarcatic dataset
df.groupby('is_sarcastic').count()

,headline,article_link
is_sarcastic,,
0,14985,14985
1,13634,13634


# 2. Retain relevant columns


Article link is not required for the computation and so dropping it. Reatining headline and is_sarcastic

In [9]:
df = df.drop('article_link',axis=1)

In [10]:
df.shape

(28619, 2)

In [18]:
df.head()

,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep. totally nails why congress is falling...
2,0,eat your veggies: 9 deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word 'strea...


# 3. Get length of each sentence


In [19]:
# Targetting headline to find max
headline_length = df['headline'].apply(len)
headline_length.max()

926

# 4. Define parameters


In [24]:
max_features = 10000
max_len = 926
embedding_size = 200

# 5. Get indices for words


In [25]:
#Creating a tokenizer
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df['headline'])

In [27]:
tokenizer.word_index

{'to': 1,
 'of': 2,
 'the': 3,
 'in': 4,
 'for': 5,
 'a': 6,
 'on': 7,
 'and': 8,
 'with': 9,
 'is': 10,
 'new': 11,
 'trump': 12,
 'man': 13,
 'at': 14,
 'from': 15,
 'about': 16,
 'by': 17,
 'after': 18,
 'you': 19,
 'this': 20,
 'out': 21,
 'up': 22,
 'be': 23,
 'as': 24,
 'that': 25,
 'it': 26,
 'how': 27,
 'not': 28,
 'he': 29,
 'his': 30,
 'are': 31,
 'your': 32,
 'just': 33,
 'what': 34,
 'all': 35,
 'who': 36,
 'has': 37,
 'will': 38,
 'report': 39,
 'into': 40,
 'more': 41,
 'one': 42,
 'have': 43,
 'year': 44,
 'over': 45,
 'why': 46,
 'day': 47,
 'u': 48,
 'area': 49,
 'woman': 50,
 'can': 51,
 's': 52,
 'says': 53,
 'donald': 54,
 'time': 55,
 'first': 56,
 'like': 57,
 'no': 58,
 'her': 59,
 'get': 60,
 'off': 61,
 'old': 62,
 "trump's": 63,
 'life': 64,
 'now': 65,
 'people': 66,
 "'": 67,
 'an': 68,
 'house': 69,
 'still': 70,
 'obama': 71,
 'white': 72,
 'back': 73,
 'make': 74,
 'was': 75,
 'than': 76,
 'women': 77,
 'if': 78,
 'down': 79,
 'when': 80,
 'i': 81,
 'my':

# 6. Create features and labels


In [29]:
X = tokenizer.texts_to_sequences(df['headline'])
X = pad_sequences(X, maxlen = max_len)
y = np.asarray(df['is_sarcastic'])

print("Number of Samples : ", len(X))
print(X[0])
print("Number of Labels : ", len(y))
print(y[0])

Number of Samples :  28619
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0

# 7. Get vocabulary size


In [30]:
num_words = len(tokenizer.word_index) + 1
num_words

30885

# 8. Create a weight matrix using GloVe embeddings


In [31]:
embeddings = {}
for o in open("DataSet - glove.6B.50d.txt"):
    word = o.split(' ')[0]
    embd = o.split(' ')[1:]
    embd = np.asarray(embd, dtype = 'float32')
    embeddings[word] = embd

In [36]:
embedding_matrix = np.zeros((num_words, 50))

for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

len(embeddings.values())

400000

# 9. Define and compile a Bidirectional LSTM model.
Hint: Be analytical and experimental here in trying new approaches to design the best model.


In [38]:
model = Sequential()
model.add(Input(shape=(max_len,)))
model.add(Embedding(num_words, 50, weights = [embedding_matrix]))
# Bi-Direction LSTM is applied
model.add(Bidirectional(LSTM(128, return_sequences = True)))
model.add(Dense(50))
model.add(Dropout(0.2))
model.add(Dense(50))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(2, activation='sigmoid'))
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 926, 50)           1544250   
                                                                 
 bidirectional (Bidirectiona  (None, 926, 256)         183296    
 l)                                                              
                                                                 
 dense (Dense)               (None, 926, 50)           12850     
                                                                 
 dropout (Dropout)           (None, 926, 50)           0         
                                                                 
 dense_1 (Dense)             (None, 926, 50)           2550      
                                                                 
 dropout_1 (Dropout)         (None, 926, 50)           0         
                                                      

In [40]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# 10. Fit the model and check the validation accuracy

In [41]:
model.fit(x_train, y_train, epochs = 5, batch_size=100, validation_split = 0.2)

Epoch 1/5
184/184 [==============================] - 845s 5s/step - loss: 0.5263 - accuracy: 0.7299 - val_loss: 0.3952 - val_accuracy: 0.8203
Epoch 2/5
184/184 [==============================] - 831s 5s/step - loss: 0.3189 - accuracy: 0.8613 - val_loss: 0.3382 - val_accuracy: 0.8565
Epoch 3/5
184/184 [==============================] - 830s 5s/step - loss: 0.2262 - accuracy: 0.9054 - val_loss: 0.3391 - val_accuracy: 0.8637
Epoch 4/5
184/184 [==============================] - 831s 5s/step - loss: 0.1565 - accuracy: 0.9376 - val_loss: 0.3700 - val_accuracy: 0.8631
Epoch 5/5
184/184 [==============================] - 826s 4s/step - loss: 0.3721 - accuracy: 0.9365 - val_loss: 0.4133 - val_accuracy: 0.8565


In [42]:
#Printing scores and accuracy
scores, accuracy = model.evaluate(x_test, y_test, verbose = 2, batch_size = 100)
print("Score :", scores)
print("Accuracy :", accuracy)

58/58 - 68s - loss: 0.3894 - accuracy: 0.8615 - 68s/epoch - 1s/step
Score : 0.38937798142433167
Accuracy : 0.8614605069160461


Overall 86% Accuracy is obtained.